In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df=pd.read_csv("security_issues.csv")
for i in df:
    print(i)
print(type(df["Stemming"][1]))
df

Issue
Number
Status
Link
Repository
Lemmatize
Stemming
<class 'str'>


,Issue,Number,Status,Link,Repository,Lemmatize,Stemming
0,"""Migrations"" -- transaction could not be decod...",3940,open,https://github.com/trufflesuite/truffle/issues...,trufflesuite/truffle,"['migrations', 'transaction', 'could', 'decode...","['migrat', 'transact', 'could', 'decod', 'coul..."
1,Deployer error: only replay-protected (EIP-155...,3935,open,https://github.com/trufflesuite/truffle/issues...,trufflesuite/truffle,"['deployer', 'error', 'replayprotected', 'eip1...","['deploy', 'error', 'replay', 'protect', 'eip'..."
2,Bug fix: Filter out fake import at end rather ...,3933,closed,https://github.com/trufflesuite/truffle/issues...,trufflesuite/truffle,"['bug', 'fix', 'filter', 'fake', 'import', 'en...","['bug', 'fix', 'filter', 'fake', 'import', 'en..."
3,Add link references to codec output when decod...,3904,closed,https://github.com/trufflesuite/truffle/issues...,trufflesuite/truffle,"['add', 'link', 'reference', 'codec', 'output'...","['add', 'link', 'refer', 'codec', 'output', 'd..."
4,Add decodeBytecode method to Truffle Decoder,3902,open,https://github.com/trufflesuite/truffle/issues...,trufflesuite/truffle,"['add', 'decodebytecode', 'method', 'truffle',...","['add', 'decodebytecod', 'method', 'truffl', '..."
...,...,...,...,...,...,...,...
21132,Go build fail on OS X: Passing char to const u...,14,closed,https://github.com/ethereum/go-ethereum/issues/14,ethereum/go-ethereum,"['go', 'build', 'fail', 'os', 'x', 'passing', ...","['go', 'build', 'fail', 'os', 'x', 'pass', 'ch..."
21133,"Update build instructions, especially for OS X",11,closed,https://github.com/ethereum/go-ethereum/issues/11,ethereum/go-ethereum,"['update', 'build', 'instruction', 'especially...","['updat', 'build', 'instruct', 'especi', 'os',..."
21134,Peer network health check,7,closed,https://github.com/ethereum/go-ethereum/issues/7,ethereum/go-ethereum,"['peer', 'network', 'health', 'check']","['peer', 'network', 'health', 'check']"
21135,Corrected typo ‘enroding > encoding’,3,closed,https://github.com/ethereum/go-ethereum/issues/3,ethereum/go-ethereum,"['corrected', 'typo', 'enroding', 'encoding']","['correct', 'typo', 'enrod', 'encod']"


In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stop_words = stopwords.words('english')

# import gensim
# from gensim.parsing.preprocessing import remove_stopwords
def remove_stopwords_fun(sentence):
    #words=sentence.split(' ')
    l=word_tokenize(sentence) # splitting into words
    f=[]
    for i in l:
        if i not in stop_words:
            f.append(i)
    s=''
    for i in f:
        s=s+i+' '
    return s
    

def clean_sentence(sentence,stopwords=False):
    sentence=sentence.lower().strip()
    sentence=re.sub(r'[^a-z0-9\s]','',sentence)
    if stopwords:
        sentence=remove_stopwords_fun(sentence)
    return sentence


def get_cleaned_sentences(df,stopwords=False):
    sents=df[['Issue']]
    cleaned_sentences=[]
    
    for  index,row in df.iterrows():
        cleaned=clean_sentence(row['Issue'],False)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences

cleaned_sentences=get_cleaned_sentences(df,stopwords=False)
#print(cleaned_sentences)

In [13]:
input_question="sign up not working"
print(input_question)

sign up not working


In [22]:
import numpy
sentences=cleaned_sentences
sentence_words=[[word for word in document.split()] for document in sentences]
from gensim import corpora
dictionary=corpora.Dictionary(sentence_words)
for key,value in dictionary.items():
    pass
#     print(key,value)
    
bow_corpus=[dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    pass
#     print(sent)
#     print(embedding)
question_org=input_question
question=clean_sentence(question_org,stopwords=False)
question_embedding=dictionary.doc2bow(question.split())

print(question,'\n','\n',question_embedding)



sign up not working 
 
 [(9, 1), (628, 1), (861, 1), (1368, 1)]


In [23]:
global dict1
def retrieve_security(question_embedding, sen_emb, faqdf, sentences):
    max_sim=-1
    index_sim=-1
    global dict1
    dict1={}
    matrix=[[0 for i in range(len(dictionary.items()))] for j in range(len(bow_corpus))]
#     for i in range(len(sen_emb)):
#         for j in sen_emb[i]:
# #             print(j[0])
#             matrix[i][int(j[0])]=1
    #print(matrix)
    in1=0
    set1=set()
    for index, embedding in enumerate(sen_emb):
#          print(sorted(question_embedding),'\n',sorted(embedding))
        
#          sim=cosine_similarity(np.array(matrix[index]).reshape(1, -1), question_embedding)
         sim=cosine_similarity(embedding,question_embedding)
         in1=in1+1
         print(index, sim, sentences[index])
         dict1.update({sentences[index]:sim})
         #list.add(sentences[index])
# #         if(sim>max_sim):
# #             max_sim=sim
# #             index_sim=index
#     #for i in sorted(dict1):
#         #print(i,dict1[i])
#     #print(dict)
#     print('\n\n')
#     print(question, df.iloc[index_sim,0])
    print(in1)


    
#retrieve_security(question_embedding, bow_corpus, df, sentences)

In [24]:
from gensim import corpora, models, similarities, downloader

In [8]:
from gensim.models import Word2Vec
import gensim.downloader as api

glove_model=None;
try:
    glove_model=gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:
    glove_model=api.load("glove-twitter-25")
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")
    

v2w_model=None;
try:
    v2w__model=gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:
    v2w_model=api.load("word2vec-google-news-300")
    v2w_model.save("./w2vecmodel.mod")
    print("Saved w2v model")
    
w2vec_embedding_size=len(v2w_model['computer'])
glove_embdding_size=len(glove_model['computer'])

Saved glove model
Saved w2v model


In [25]:
def getWordVec(word,model):
    samp=model['computer']
    vec=[0]*len(samp)
    try:
        vec=model[word]
    except:
        vec=[0]*len(samp)
    return (vec)

def getPhraseEmbedding(phrase,embeddingmodel):
    samp=getWordVec('computer', embeddingmodel)
    vec=numpy.array([0]*len(samp))
    den=0
    for word in phrase.split():
        den=den+1
        vec=vec+numpy.array(getWordVec(word,embeddingmodel))
    return vec.reshape(1,-1)
    

In [26]:
#with Glove

sent_embeddings=[]

for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,glove_model))
    
question_embedding=getPhraseEmbedding(question,glove_model)

retrieve_security(question_embedding,sent_embeddings,df,cleaned_sentences)

0 [[0.95230924]] migrations  transaction could not be decoded could not recover secp256k1 key calculated rx is larger than curve p
1 [[0.89947758]] deployer error only replayprotected eip155 transactions allowed over rpc
2 [[0.96699801]] bug fix filter out fake import at end rather than start fix compile errors
3 [[0.86996531]] add link references to codec output when decoding bytecode
4 [[0.82456274]] add decodebytecode method to truffle decoder
5 [[0.92065199]] internal improvement add new compatible nativize function and some missing codecdecoder functions
6 [[0.43630279]] truffle db  hardhat integration poc
7 [[0.9199492]] add missing shebang to truffledbkit and chmod x
8 [[0.9171605]] revert internal improvement add compatiblenativize function and some missing codecdecoder functions
9 [[0.8754426]] create truffledbkit components for commonly used patterns like collections of spinners
10 [[0.90773945]] problem with stacktraces and optimized code in solidity 08x
11 [[0.82718257]] al

453 [[0.81735762]] test file whitelist filter out irrelaventtemporary files fix 40
454 [[0.8980586]] closes 41 makes injectcontracts postprocessor work in node env
455 [[0.9252788]] injectcontracts postprocessor dont work in node environment
456 [[0.95488208]] not able to open eter wallet
457 [[0.94998554]] no passwordpassphrase how to get imported to other wallet
458 [[0.95297878]] cannot transfer my ethereum out of wallet to ledger nano s
459 [[0.9471284]] please help cant open my eth wallet
460 [[0.97229244]] me ethereum wallet stopped syncing with the network on new years day during the last upgrade and its been 6 days and it wont sync at all i live in cincinnati ohio area  usa
461 [[0.97774679]] how much time it takes to generate wallet
462 [[0.50379593]] checksum mismatch in dowloaded node
463 [[0.90488365]] wallet problem geth finishes work
464 [[0.95382948]] how to update  wallet
465 [[0.84805254]] wallet problem
466 [[0.90241826]] unable to launch ethereum wallet v0111
467 [[0

968 [[0.95920795]] cant see balance in ethereum wallet
969 [[0.96549453]] the ethereum have not reached the wallet
970 [[0.77808334]] cannot add eos token wallet freezes
971 [[0.95149585]] my eth did not sent to my mist wallet
972 [[0.94747614]] how to import a existing account file when i install the ethereum wallet with gnulinux
973 [[0.95573953]] ethereum sent from coinbase and confirmed but still not received in wallet
974 [[0.95296862]] 1 go to the assets tab in your imtoken wallet
975 [[0.93441578]] my password is wrong
976 [[0.87977767]] wallet contract notifying a lot of transactions
977 [[0.93828732]] wallet downloading to c even though i chosed partition e for block data
978 [[0.96415561]] i cant  complete create a wallet contract
979 [[0.95024783]] ether is not shown in the wallet
980 [[0.95773192]] ethereum wallet is looking for the wrong geth version number
981 [[0.91974034]] missing eth from my wallet
982 [[0.88393565]] wallet will not sync  sits at ipc endpoint opened
98

1478 [[0.94063778]] cant withdraw from ethereum wallet 074 mac
1479 [[0.71800467]] readme correct checksums add npmupdate
1480 [[0.92271723]] ethereum wallet account gives wrong password message but password never changed please help
1481 [[0.8886093]] random transactions in wallet list
1482 [[0.87738092]] no md5 checksum on 075 release
1483 [[0.82268964]] ethereum wallet frozen on splash screen
1484 [[0.78852401]] fixed wallet tab insertion added admin permission
1485 [[0.88966204]] ethereum wallet 075 freezes after loading up
1486 [[0.88185519]] rename ethereumwallet to ethereum wallet
1487 [[0.95494454]] wallet not opening
1488 [[0.87382925]] missing transactions on wallet contracts
1489 [[0.92429135]] copy address to clipboard not functioning
1490 [[0.94046111]] option to let mist use a remote getheth instance on another machine
1491 [[0.96339361]] being asked for password to send from etherbase account on mist  never made password
1492 [[0.95137249]] avast deleted a virus it found

1962 [[0.90647666]] metamask desktop notifications still link to etherscan when using a custom rpc with a custom block explorer
1963 [[0.93975773]] get metamask rate from quote objects instead of fee endpoint
1964 [[0.96152164]] update viewquote designs to better represent the metamask fee
1965 [[0.92034805]] metamask ethaccounts unexpectedly updated accounts please report this bug
1966 [[0.90437908]] fix 9577  use contained button type for unlock button
1967 [[0.95875799]] fix exception upon failure to get next nonce
1968 [[0.91114715]] react error on login screen  failed prop type invalid prop variant of value raised supplied to forwardrefbutton
1969 [[0.92790658]] use chainid values for the address book
1970 [[0.95641091]] prevent undefined class from being added to tabs
1971 [[0.89519316]] update the metaswap contract address
1972 [[0.91149893]] choosing metamask notification item in history menu kills window
1973 [[0.83858867]] fix 9516  prevent duplicate incrementdecrement button

...

18970 [[0.93915536]] discard child block with parenthash not matching hash of imported block
18971 [[0.52107055]] verify contracts rpc
18972 [[0.93938165]] force consensus loss for parent block if its hash mismatches parenthash
18973 [[0.94979948]] unknown action when getting account balance for a specific address eth mainnet
18974 [[0.86582934]] add petersburg evm version to smart contract verifier
18975 [[0.82639321]] optimize token holder count updates when importing address current balances
18976 [[0.8762257]] infinity timeout for addresses without code query
18977 [[0.92766092]] use code fetcher for failed created addresses to properly employ batching
18978 [[0.95145815]] do not fetch burn address balance
18979 [[0.79396156]] feat rpc endpoint to list addresses
18980 [[0.89218526]] fix usd value on address page
18981 [[0.75859199]] cardinality violation while importing blockrewards
18982 [[0.85577274]] verify smart contract through an api endpoint
18983 [[0.81146642]] encode integ

19442 [[0.89619261]] store updates log event source of removal operations and make string formatting of block hashes consistent
19443 [[0.89475058]] fix bugs in logging removal operations and in string formatting of block hashes
19444 [[0.62171485]] remove access tokens  authorization
19445 [[0.89803966]] update rustasc type conversions improve signed and unsigned bigint handling
19446 [[0.96383933]] check for underoverflow when reading bigints from asc as u256
19447 [[0.94190973]] use address field in ethgetlogs
19448 [[0.93329923]] check block hash on old receipts
19449 [[0.93518556]] travis fix and refactor style check and release building
19450 [[0.94375366]] feature  fields for both mainnet and testnet addresses in subgraphyaml
19451 [[0.91404353]] create integration test with a complete example subgraph
19452 [[0.97419103]] scanning blocks for common event signatures like transfer takes too long
19453 [[0.65607084]] allow dynamically addingremoving tracked contract addresses
1945

19933 [[0.81147257]] sync panic runtime error invalid memory address or nil pointer dereference downloaderfindancestor
19934 [[0.8219453]] trie improve range proof
19935 [[0.89415656]] accountsexternal convert signature v value to 01
19936 [[0.90002664]] clef ethsign returns incorrect signature when geth is run with signer
19937 [[0.91915746]] core avoid doublelock in txpooltest
19938 [[0.92881472]] accounts add blockbynumbernolock to avoid doublelock
19939 [[0.95667085]] core add skip in addchildindexer to avoid double lock
19940 [[0.78341364]] invalid memory address 1913 windows
19941 [[0.64279988]] access violation 1913exit
19942 [[0.84833646]] accountskeystore fix vulnerability in zerokey
19943 [[0.60889341]] cmdclef signercore password input fixes
19944 [[0.8658085]] snapshot add unlock before returnpanic
19945 [[0.93435372]] event fix inconsistency in lock and unlock
19946 [[0.92846673]] core fix the condition of reorg
19947 [[0.93557093]] core the conditions of reorg are differe

20419 [[0.94133352]] send raw transaction return rlp input string too long for commonaddress decoding into typestransactiontypestxdatarecipient
20420 [[0.93667165]] allow any vhost for wallet deployed by puppeth
20421 [[0.8533449]] geth console special characterencoding problem
20422 [[0.85698184]] puppeth  cant deploy faucet to remote machine
20423 [[0.95833221]] change the md5 checksum from geth 182 it is not downloaded automatically ethereum wallet
20424 [[0.84529664]] hash mismatch from httpsgethstoreblobcorewindowsnet
20425 [[0.95188603]] a feedback about the cheat and fraud of etc eth on twitter
20426 [[0.95919513]] go version check fails for go  110
20427 [[0.92128586]] core flush trie db if memory data if limit condition meets
20428 [[0.89155112]] whisper serious security issue fixed
20429 [[0.88198119]] whisper mailserver no longer supports the signature vaidation
20430 [[0.82459335]] face authentication in ubuntu 1710
20431 [[0.92652361]] puppeth  allow to switch between ssh 

20920 [[0.89699443]] removed tx from pool low tx nonce
20921 [[0.69851738]] gpo nonexistent block checks
20922 [[0.94327755]] not able to kill geth process on mac os x  zombie
20923 [[0.87639606]] unable to unlock account created with plaintext password in geth
20924 [[0.83967785]] race condition in http interface
20925 [[0.88533253]] upgradedb nonce is invalid got 42
20926 [[0.20795283]] sec50 rlpx aes ctr keystream reusage
20927 [[0.90686451]] ethfetcher handle and test various dos attacks
20928 [[0.90686451]] ethfetcher handle and test various dos attacks
20929 [[0.94425043]] transaction hash is not available to the user on contract creation
20930 [[0.95962261]] ethdownloader may be stuck after a batch of duplicate hashes
20931 [[0.64194014]] contractinfoget content hash mismatch
20932 [[0.89321291]] contract eventwatch now spams console with trues and falses leftin debug
20933 [[0.72779342]] transaction w invalid nonce
20934 [[0.8483207]] integrate console and remove old rpc packag

In [21]:
import operator
cd1 = sorted(dict1.items(),key=operator.itemgetter(1),reverse=True)
for i in cd1:
    print(i)

('waitfortransaction can be faster like it used to be by checking immediately', array([[0.98695826]]))
('im on attempt 9 at just getting the wallet to simply launch losing my sanity', array([[0.98632194]]))
('i can not get contract address when call contractfactorydeploy', array([[0.98547694]]))
('all peers dropping and getting stuck at cant connect to address', array([[0.98538398]]))
('i am syncing last 10 days to find ethereum in my wallet started 1000x and number of blocks that need to sync always goes up so can not be reached please help', array([[0.98500327]]))
('ethereumselectedaddress is not reset after sign out', array([[0.98453327]]))
('better resync checking and running', array([[0.98425781]]))
('ethereum wallet wont sync been trying to load it for 4 days it starts and then stops any way to fix this', array([[0.98425347]]))
('check to make sure an app doesnt have a running worker instance before starting one', array([[0.98412882]]))
('erc20 tokens wont show up view on ethersc

('now with clientside signing', array([[0.95725317]]))
('how to create and signature a transaction manually like ethereumgoethereum', array([[0.95725273]]))
('web3ethsubscribe not working for account addresses', array([[0.95724377]]))
('slow sign in unlock accounts db before starting node', array([[0.95723044]]))
('javascript heap out of memory when trying to verify a contract', array([[0.95720428]]))
('transaction card address is wrapping to the next line', array([[0.95719533]]))
('i cannot open my metamask', array([[0.95718625]]))
('gui doesnt start gui asset not found you can set an alternative asset path on the command line using option assetpathopen usrsharemistqmlwalletqml no such file or directory', array([[0.9571862]]))
('add fallback for case where the contract address is lowercased', array([[0.95715423]]))
('eth address does not work', array([[0.95715264]]))
('how do i call enssetupowner with a signed trnasaction', array([[0.9571094]]))
('sending ether to contract using metam

('8657 keyboard overlaps sign button if you tap on sign after changing recipient or asset field on send transaction screen', array([[0.94757731]]))
('keyboard overlaps sign button if you tap on sign after changing recipient or asset field on send transaction screen', array([[0.94757731]]))
('fixed several case ids and add test for checking loglevel and fleet', array([[0.9475712]]))
('restore my account from the 12 words mnemonic phrase but get new account with new address', array([[0.94756364]]))
('shasum 256 checksums not correct for osx', array([[0.94755318]]))
('imported wallet contract gets stuck every time in importing status', array([[0.94755074]]))
('100beta55 terminal hangs forever and doesnt return the contract address after contract deploy', array([[0.94753574]]))
('accept list or string for filterparam address', array([[0.94753463]]))
('if the keystore folder has duplicate keys during accounts import itll exit with fatal error', array([[0.94752936]]))
('make parity work with

('fix encoding of ecdsa signatures and public keys', array([[0.93549334]]))
('signer blocks address is already in use message', array([[0.9354924]]))
('change input argument size in delegation call and remove useless code', array([[0.93549017]]))
('hide all controls when staking is forbidden by contract', array([[0.93548247]]))
('small bug with enable with privacy mode off', array([[0.93547716]]))
('unable to transfer to ethereumjswallet address', array([[0.93547392]]))
('ethapi private tx bug fixes for sign and send apis', array([[0.93546099]]))
('api add debugaccountexist that checks whether an account exists', array([[0.93545844]]))
('convert gasfeedisplayv2js to follow pattern in the send directory', array([[0.9354179]]))
('get a new wallet fix json  keystore', array([[0.93541043]]))
('codegen this commit removes one of duplicate assignments', array([[0.93540079]]))
('unable to generate same account from metamask mnemonic', array([[0.93539707]]))
('get a new wallet page spelling is

('encoding of data field under contracts is wrong', array([[0.92263659]]))
('impossible to withdraw zrx from metamask to etherdelta', array([[0.92263144]]))
('enhance view address screen with a dropdown that draws from address book and recent addresses', array([[0.92262606]]))
('add 0x prefix to the logfilteraddress', array([[0.92262041]]))
('wrong password in 075', array([[0.92261226]]))
('7671 remove ability to change values while signing transaction', array([[0.92261032]]))
('add mythril to list of security tools', array([[0.92260883]]))
('update the walletsol to include new updated code', array([[0.92259618]]))
('parity 140  copy and paste not working correctly', array([[0.92259397]]))
('allow zeropass to wallets in development mode', array([[0.92258233]]))
('tests updated tests for deleting public chat and wallet recovery', array([[0.92257763]]))
('remove audits and resolve everything for testing', array([[0.92257083]]))
('new modifier for reentrancysafe functions which denies cal

...

('update privacy and terms articles', array([[0.87192854]]))
('initial integration with gorhoms bottom sheet on android', array([[0.87192067]]))
('allow accessing content at apicontenthash', array([[0.87191835]]))
('use async in download wallet', array([[0.87191772]]))
('unable to connect parity blocked connection due to invalid token', array([[0.8719116]]))
('peer address and port fixes', array([[0.8719088]]))
('add integration tests to cover trinity attach web3 interaction', array([[0.87189715]]))
('update sad wallet image', array([[0.87187905]]))
('new vps address for precompiled dependencies on master branch', array([[0.87187463]]))
('display full transaction hash on new transaction', array([[0.87183359]]))
('network specification ready for audit', array([[0.87183307]]))
('updated paritylocalstore to edition 2018', array([[0.87182878]]))
('update mailserver addresses for whisper v6 in constantscljs', array([[0.87182707]]))
('verify block number consistency', array([[0.87180919]]))


('address pr feedback in black demo pr', array([[0.84541174]]))
('problem verifying personalsign signatures produced by ledger v5', array([[0.84541006]]))
('fix selfdestruct revert bug', array([[0.84540837]]))
('mistethereum wallet as a exefile', array([[0.84540023]]))
('trust', array([[0.84539364]]))
('add ios permissions checking on transition', array([[0.84537121]]))
('fix signtypeddata bytes signing', array([[0.84537115]]))
('integrate with whisper ver 5', array([[0.84536792]]))
('update error usage patterns to go113', array([[0.8453352]]))
('token registering name in parity', array([[0.84532691]]))
('sideeffect restriction on different stages', array([[0.8453112]]))
('handle transaction decoding in trufflecodec', array([[0.84529775]]))
('hash mismatch from httpsgethstoreblobcorewindowsnet', array([[0.84529664]]))
('doc says functions are by default internal  parity exploit', array([[0.84526857]]))
('recommendations for preventing address switching attacks', array([[0.84526159]]))


('cmdpuppeth integrate istanbul into puppeth', array([[0.80403945]]))
('core coretypes miner fix transaction nonceprice combo sort', array([[0.80402926]]))
('possible to configure frontend api through ssl https', array([[0.80401742]]))
('fix verify contract loading button width', array([[0.80398131]]))
('migrate test deps job to npm audit', array([[0.80392909]]))
('reverse call data decoding given transaction data  method', array([[0.80391496]]))
('document the insufficient gas griefing attack', array([[0.80390685]]))
('add parity jobs to circleci config', array([[0.80385943]]))
('parity cli exits with error during startup', array([[0.80379908]]))
('enode address library', array([[0.80378423]]))
('duplicated attestations in db', array([[0.80372314]]))
('parity not sync blocks', array([[0.803722]]))
('runtimewasm add hosted abi encodedecode functions', array([[0.80371752]]))
('fix audit issues', array([[0.803674]]))
('enable codesigning on osx build', array([[0.80358619]]))
('geth defau

('update wallet library binaries', array([[0.73620008]]))
('sha3 remove encoding param old primitive logic', array([[0.73619148]]))
('rejecting metamask transaction sends ridiculous error notification', array([[0.73605873]]))
('stack backtrace 18 parity keeps crashing', array([[0.73603206]]))
('event overloading with dynamic arrays are encoded incorrectly', array([[0.73601894]]))
('named verified event parameter', array([[0.73600932]]))
('breaks norwegian bankid authentication system', array([[0.73599149]]))
('geth crash with signal sigsegv segmentation violation code0x1 addr0x0 pc0x23e04', array([[0.73592659]]))
('integrate codegolfing improvements remove nonstandard related utility code', array([[0.7359264]]))
('consider decode failures as reverts', array([[0.73587495]]))
('assembly strict mode', array([[0.73582509]]))
('proceeding with npm audit', array([[0.73570979]]))
('parity not syncing', array([[0.73567116]]))
('parity 183186 musicoin not syncing', array([[0.73567116]]))
('reve

('login per site ui', array([[0.57494568]]))
('humanize ens address error', array([[0.57491921]]))
('merged address components', array([[0.57489516]]))
('miscellaneous security improvements', array([[0.57442691]]))
('metamask tsx issues', array([[0.57431111]]))
('metamask fixes patch', array([[0.57425576]]))
('indexertokenbalancefetcher duplicated token balance error', array([[0.57416737]]))
('smtchecker fix chc encoding', array([[0.57358222]]))
('tracking  integrating slashing protection invariants', array([[0.57341804]]))
('revert transaction queue integration', array([[0.57308193]]))
('nonce tracking error  web3ethsendtransaction incorrect nonce', array([[0.57306251]]))
('implement slashing conditions', array([[0.57300601]]))
('no etherum httpsblockchaininfoaddress1crnsdjtjyhbwwvaynkhufmystefsmnnlh', array([[0.57287457]]))
('compact signature representation', array([[0.57262588]]))
('draft implement ewasm verifier', array([[0.57228947]]))
('disabling wallet', array([[0.57195808]]))
